In [1]:
import shapely

### parameters
place = 'tel_aviv'
feature = 'green_canopy'

In [27]:

from rasterio.features import geometry_mask
import rasterio

from rasterio.warp import calculate_default_transform, reproject, Resampling
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping


from tqdm import tqdm  # for progress bar
import matplotlib.pyplot as plt





import numpy as np
import os
from pathlib import Path

import warnings
warnings.filterwarnings(action='ignore')
crs_prj = 'EPSG:2039'


# Get the current working directory (e.g., the folder you're running from)
cwd = Path().resolve()

# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)
os.makedirs(f'{parent_folder}/shp/{feature}',exist_ok=True)
detail_folder = f'{data_folder}/{feature}'

In [24]:



# --- Load bands and compute NDVI ---
with rasterio.open(f'{detail_folder}/B04.tiff') as red_src, \
     rasterio.open(f'{detail_folder}/B08.tiff') as nir_src:
    
    red = red_src.read(1).astype('float32')
    nir = nir_src.read(1).astype('float32')
    meta = red_src.meta.copy()

# Compute NDVI and threshold for healthy vegetation
ndvi = (nir - red) / (nir + red)
ndvi = np.clip(ndvi, -1, 1)
healthy = ndvi > 0.3  # Adjust threshold as needed

# Save binary healthy vegetation raster
meta.update(dtype=rasterio.float32, count=1, driver='GTiff')
with rasterio.open(f'{detail_folder}/healthy.tiff', 'w', **meta) as dst:
    dst.write(healthy.astype(rasterio.float32), 1)

# --- Reproject raster to desired CRS (e.g., ITM) ---
src_path = f'{detail_folder}/healthy.tiff'
reprojected_path = f"{detail_folder}/healthy_itm.tiff"

with rasterio.open(src_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, crs_prj, src.width, src.height, *src.bounds)
    
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': crs_prj,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(reprojected_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=crs_prj,
                resampling=Resampling.nearest)

# --- Analyze green cover per street ---
# Load and buffer streets
streets_raw = gpd.read_file(f'{data_folder}/streets.shp')[['oidrechov', 'geometry']]
streets = streets_raw.to_crs(crs_prj)
streets['geometry'] = streets.geometry.buffer(25)

# Load reprojected raster
with rasterio.open(reprojected_path) as src:
    raster = src.read(1)
    raster_transform = src.transform

# Calculate green pixel percentage for each street
results = []
for _, row in tqdm(streets.iterrows(), total=len(streets)):
    geom = row.geometry

    mask = geometry_mask([mapping(geom)],
                         transform=raster_transform,
                         invert=True,
                         out_shape=raster.shape)

    masked_values = raster[mask]
    total_pixels = masked_values.size
    green_pixels = np.sum(masked_values == 1)

    percent_green = (green_pixels / total_pixels) if total_pixels > 0 else -1

    results.append({
        'oidrechov': row['oidrechov'],
        'green_pixels': int(green_pixels),
        'total_pixels': int(total_pixels),
        'green_percent': round(percent_green, 2),
        'geometry': geom
    })

# --- Merge and Export Results ---
results_df = pd.DataFrame(results)
streets_result = streets_raw.merge(results_df, on='oidrechov')

# Save to shapefile
streets_result.to_file(f"{detail_folder}/{feature}.shp")

# Preview results
print(streets_result[['oidrechov', 'green_percent']].head())
